In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re


DATA_DIR = 'data/'
WIKIPEDIA_BASE_URL = 'https://fr.wikipedia.org'

In [27]:
def parse_url(url):
    page = urllib.request.urlopen(url).read().decode('utf-8')
    return BeautifulSoup(page, 'html.parser')

def get_table_from_article(url_or_soup, table_index, col_indices, col_names, col_links_indices, col_links_names):
    soup = parse_url(url_or_soup) if isinstance(url_or_soup, str) else url_or_soup
    table = soup.find_all('table', class_='sortable')[table_index]
    data = []
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        cols = [td.get_text(',', strip=True) for td in tds]
        cols = [cols[i] for i in col_indices]
        links = []
        for index in col_links_indices:
            link = tds[index].find('a', href=True)
            if link:
                link = link['href']
                if link.endswith('&action=edit&redlink=1'):
                    link = None
                elif link.startswith('/'):
                    link = WIKIPEDIA_BASE_URL + link
            else:
                link = None
            links.append(link)
        data.append((*cols, *links))
    df = pd.DataFrame(data, columns=[*col_names, *col_links_names])
    return df

def extract_monuments_list(article_or_url, col_indices=None):
    col_indices = [0, 4, 5] if not col_indices else col_indices
    df2 = get_table_from_article(article_or_url, table_index=0, col_indices=col_indices,
                                 col_names=['Name', 'Status', 'Date'],
                                 col_links_indices=[0], col_links_names=['Link'])
    return df2[(df2.Status.str.contains('Classé')) & (df2.Link)]

In [3]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_par_commune_fran%C3%A7aise'
df = get_table_from_article(url, table_index=1, col_indices=[1, 4], col_names=['Commune', 'Number of monuments'],
                            col_links_indices=[1], col_links_names=['Link'])
df.head(3)

,Commune,Number of monuments,Link
0,Paris,1823,https://fr.wikipedia.org/wiki/Liste_des_monume...
1,Bordeaux,365,https://fr.wikipedia.org/wiki/Liste_des_monume...
2,La Rochelle,292,https://fr.wikipedia.org/wiki/Monuments_de_La_...


In [4]:
monuments_per_commune = {}
for _, row in df.iterrows():
    article = parse_url(row.Link)
    keywords = ['Liste', 'Monuments historiques', 'Immobiliers', 'Immobilier', 'Monuments actuels',
                'Monuments immobiliers', 'Liste exhaustive', 'Caen']
    found_keywords = [article.find('span', class_='mw-headline', text=keyword) for keyword in keywords]
    if sum(map(bool, found_keywords)):
        monuments_per_commune[row.Commune] = extract_monuments_list(article)
    else:
        print('Skipping {} (<{}>)'.format(row.Commune, row.Link))

Skipping Paris (<https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Paris>)
Skipping La Rochelle (<https://fr.wikipedia.org/wiki/Monuments_de_La_Rochelle>)


In [5]:
monuments_per_commune['Bordeaux'].head(3)

,Name,Status,Date,Link
0,Abbatiale Sainte-Croix,Classé,1840,https://fr.wikipedia.org/wiki/Abbatiale_Sainte...
1,Ancien Hôtel de ville de Bordeaux,Classé,1886,https://fr.wikipedia.org/wiki/Grosse_cloche_de...
2,Basilique Saint-Michel,Classé,1846,https://fr.wikipedia.org/wiki/Basilique_Saint-...


In [6]:
urls = ['https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_1er_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_2e_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_3e_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_4e_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_5e_arrondissement_de_Paris']
for i, url in enumerate(urls):
    name = 'Paris' + str(i)
    monuments_per_commune[name] = extract_monuments_list(url)

In [7]:
monuments_per_commune['Paris1'].head(3)

,Name,Status,Date,Link
2,Basilique Notre-Dame-des-Victoires,Classé,1972,https://fr.wikipedia.org/wiki/Basilique_Notre-...
3,Bibliothèque nationale de France,Classé,1983,https://fr.wikipedia.org/wiki/Biblioth%C3%A8qu...
10,Église Notre-Dame-de-Bonne-Nouvelle,Classé,1983,https://fr.wikipedia.org/wiki/%C3%89glise_Notr...


In [28]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_La_Rochelle'
monuments_per_commune['La Rochelle'] = extract_monuments_list(url)

url = 'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Lyon'
monuments_per_commune['Lyon'] = extract_monuments_list(url, col_indices=[0, 5, 6])

In [10]:
monuments = pd.concat({k: v.reset_index(drop=True) for k, v in monuments_per_commune.items()})
monuments.loc['Abbeville']

,Name,Status,Date,Link
0,Carrière Carpentier,Classé,1983,https://fr.wikipedia.org/wiki/Carri%C3%A8re_Ca...
1,Carrière de Menchecourt,Classé,1983,https://fr.wikipedia.org/wiki/Carri%C3%A8re_de...
2,Église Notre-Dame-de-la-Chapelle,Classé,1910,https://fr.wikipedia.org/wiki/%C3%89glise_Notr...
3,Église Saint-Sépulcre,Classé,1907,https://fr.wikipedia.org/wiki/%C3%89glise_Sain...
4,Église Saint-Vulfran,Classé,1840,https://fr.wikipedia.org/wiki/%C3%89glise_Sain...
5,Manufacture des Rames,"Inscrit,Classé","1984,1986",https://fr.wikipedia.org/wiki/Manufacture_des_...


In [11]:
monuments.to_csv(DATA_DIR + '_monuments.csv')

In [29]:
df3 = pd.read_csv(DATA_DIR + '_stations.csv')
communes = [v for v in monuments.index.levels[0].values]
unknowns = set(communes) - set(df3[df3.Name.isin(communes)].Name)

In [40]:
monuments_per_commune['Crémieu'] = pd.DataFrame.from_dict([{'Name': '', 'Status': '', 'Date': '', 'Link': ':-('}])
for unknown in unknowns:
    print(unknown)
    print(monuments_per_commune[unknown].Link.iloc[0])

Nancy
https://fr.wikipedia.org/wiki/Arc_H%C3%A9r%C3%A9
Île-d'Aix
https://fr.wikipedia.org/wiki/%C3%89glise_Saint-Martin_d%27Aix
Montpellier
https://fr.wikipedia.org/wiki/Arc_de_triomphe_(Montpellier)
Le Faou
https://fr.wikipedia.org/wiki/%C3%89glise_Notre-Dame_de_Rumengol
Saumur
https://fr.wikipedia.org/wiki/Abbaye_Saint-Florent_de_Saumur
Fort-de-France
https://fr.wikipedia.org/wiki/Biblioth%C3%A8que_Sch%C5%93lcher
Paris3
https://fr.wikipedia.org/wiki/Biblioth%C3%A8que_de_l%27Arsenal
Collonges-la-Rouge
https://fr.wikipedia.org/wiki/%C3%89glise_Saint-Pierre_de_Collonges-la-Rouge
Versailles
https://fr.wikipedia.org/wiki/B%C3%A2timents_du_Man%C3%A8ge
Dijon
https://fr.wikipedia.org/wiki/Abbaye_Saint-B%C3%A9nigne_de_Dijon
Riquewihr
https://fr.wikipedia.org/wiki/Abbaye_d%27Autrey
Bourg-Saint-Andéol
https://fr.wikipedia.org/wiki/%C3%89glise_Saint-And%C3%A9ol_de_Bourg-Saint-And%C3%A9ol
Lyon
https://fr.wikipedia.org/wiki/Amphith%C3%A9%C3%A2tre_des_Trois_Gaules
Marseille
https://fr.wikipedia.org

In [43]:
print(set(df3[df3.Name.isin(communes)].Name))

{'Briançon', 'Bayonne', 'Grenoble', 'Blois', 'Arras', 'Angoulême', 'Béziers', 'Grasse', 'Niort', 'Tarascon', 'Nîmes', 'Saintes', 'Salins-les-Bains', 'Sélestat', 'Fréjus', 'Verdun', 'Perpignan', 'Thiers', 'Arles', 'Auray', 'Haguenau', 'Mende', 'Laval', 'Chartres', 'Poitiers', 'Vannes', 'Lisieux', 'Troyes', 'Tournus', 'Nantes', 'Tours', 'Amboise', 'Obernai', 'Gray', 'Pézenas', 'Albi', 'Dieppe', 'Vichy', 'Bourg-en-Bresse', 'Provins', 'Montbéliard', 'Honfleur', 'Alençon', 'Wissembourg', 'Beauvais', 'Brive-la-Gaillarde', 'Pont-à-Mousson', 'Chalon-sur-Saône', 'Rennes', 'Chinon', 'Colmar', 'Châtillon-sur-Seine', 'Orléans', 'Fontenay-le-Comte', 'Villeneuve-lès-Avignon', 'Cluny', 'Valenciennes', 'Autun', 'Dunkerque', 'Carpentras', 'Reims', 'Le Havre', 'Quimper', 'Roubaix', 'Aire-sur-la-Lys', 'Clermont-Ferrand', 'Amiens', 'Langres', 'Senlis', 'Bar-le-Duc', 'Dinan', 'Cahors', 'Loches', 'Sens', 'Épinal', 'Bourges', 'Soissons', 'Phalsbourg', 'Bayeux', 'Compiègne', 'Carcassonne', 'Fougères', 'Périgu

In [77]:
url = 'https://fr.wikipedia.org/wiki/Villes_et_Pays_d%27art_et_d%27histoire'
df4 = get_table_from_article(url, table_index=0, col_indices=[0, 2], col_names=['Name', 'Number of communes'],
                            col_links_indices=[0], col_links_names=['Link'])
df4.loc[df4['Number of communes'] == '', 'Number of communes'] = '99'
df4['Number of communes'] = df4['Number of communes'].apply(lambda x: int(re.sub(',\[,Note.*', '', x)))
df4 = df4[(df4['Number of communes'] == 1) & (df4.Link)]

In [78]:
df4

,Name,Number of communes,Link
1,Laon,1,https://fr.wikipedia.org/wiki/Laon
2,Saint-Quentin,1,https://fr.wikipedia.org/wiki/Saint-Quentin
3,Soissons,1,https://fr.wikipedia.org/wiki/Soissons
4,Moulins,1,https://fr.wikipedia.org/wiki/Moulins_(Allier)
6,Briançon,1,https://fr.wikipedia.org/wiki/Brian%C3%A7on
7,Grasse,1,https://fr.wikipedia.org/wiki/Grasse
8,Menton,1,https://fr.wikipedia.org/wiki/Menton_(Alpes-Ma...
11,Charleville-Mézières,1,https://fr.wikipedia.org/wiki/Charleville-M%C3...
12,Sedan,1,https://fr.wikipedia.org/wiki/Sedan
14,Troyes,1,https://fr.wikipedia.org/wiki/Troyes
